## 1. Изучаем датасет слов с размеченной тональностью 
#### Взято из репозитория https://github.com/dkulagin/kartaslov
## 2. Предобрабатываем данные под свои цели
## 3. Пишем простую модель классификации эмоциональной тональности текста

In [ ]:
!pip3 install pandas

In [1]:
import pandas as pd

### Подгружаем датасет

In [13]:
words_df = pd.read_csv('kartaslovsent.csv', encoding = 'utf-8', sep=';')

### Смотрим размерность датасета и какие есть колонки. 
#### (результат: к нашим услугам 46127 слов с размеченной эмоциональной тональностью)

In [4]:
words_df.shape

(46127, 8)

### Cмотрим, как выглядят данные
#### (результат: Видим, что помимо PSTV (позитивные) и NGTV (негативные) имеются нейтральные слова размеченные как NEUT

In [14]:
words_df.head(20)

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19
5,абитуриент,NEUT,0.23,0.235,0.049,0.500,0.216,0.00
6,абитуриентка,NEUT,0.34,0.294,0.029,0.461,0.216,0.00
7,абонемент,NEUT,0.18,0.232,0.056,0.560,0.152,0.00
8,абонементный,NEUT,0.19,0.210,0.070,0.500,0.220,0.00
9,абонент,NEUT,0.00,0.075,0.075,0.675,0.175,0.00


### Выбираем из датасета только слова позитивно и негативно размеченные. 
### Очищаем данные от ненужного, то есть оставляем колонки term, tag, pstv, ngtv, neut. Смотрим размерность

In [6]:
words_df = words_df[['term', 'tag', 'pstv', 'ngtv', 'neut']]

negative_words = words_df[words_df['tag'] == 'NGTV']
negative_words = negative_words.sort_values(by='ngtv', ascending = False)
positive_words = words_df[words_df['tag'] == 'PSTV']
positive_words = positive_words.sort_values(by='pstv', ascending = False)
neutral_words = words_df[words_df['tag'] == 'NEUT']
neutral_words = neutral_words.sort_values(by='neut', ascending = False)

print(f'Размерность датасета: {words_df.shape[0]}')
print(f'Слов негативно окрашенных в датасете:{negative_words.shape[0]}')
print(f'Слов позитивно окрашенных в датасете:{positive_words.shape[0]}')
print(f'Слов нейтрально окрашенных в датасете:{neutral_words.shape[0]}')

Размерность датасета: 46127
Слов негативно окрашенных в датасете:11863
Слов позитивно окрашенных в датасете:6215
Слов нейтрально окрашенных в датасете:28049


### Пишем простой распознаватель эмоциональной тональности текста

In [10]:
def emotion_recognizer(text):
    '''Принимает текст, оценивает его эмоциональную окраску.
    Возвращает positive/negative/neutral'''
    
    negative_color, positive_color, neutral_color = 0, 0, 0
    text = text.lower()
    text = text.strip(',').split(' ')
    for word in text:
        if len(word) < 3: #чтобы исключить союзы и предлоги
            continue
        for check_word in list(negative_words['term']):
            if word in check_word: 
                negative_color +=1
                continue
        for check_word in list(positive_words['term']):
            if word in check_word: 
                positive_color +=1
                continue
        for check_word in list(neutral_words['term']):
            if word in check_word: 
                neutral_color +=1
                continue
                
    if negative_color > positive_color and negative_color > neutral_color:
        return 'negative'
    elif positive_color > negative_color and positive_color > neutral_color:
        return 'positive'
    elif neutral_color > negative_color and neutral_color > positive_color:
        return 'neutral'
    

### Тестируем

In [8]:
test_comments = [
    {'user_name':'Gvido',
    'comment':'Помогите Автономные тесты. Интеграционные, функциональные, unit-тесты Все давно уже знают, что такое MVC и согласны с тем, что это хорошо. Но откуда тогда берутся эти проекты с километровым кодом в контроллерах, да ещё и напрямую изменяющим БД? Почему это плохо и как должно быть, а также о главной боли Django — в этом выпуске.Посоветуйте курс по пайтону, реальный - не поверхностный. Реально хочу погрузиться в программирование. В этом видео мы реализуем Телеграм бота для учёта личных расходов с бюджетом, категориями расходов и базовой статистикой. Использовать будем Python и Aiogram с Asyncio, в качестве базы данных легковесный движок SQLite, а результат работы упакуем в Docker контейнер для быстрого разворачивания на любом сервере'},
    {'user_name':'luiza mariam_',
    'comment':'я с прошлой субботы жду информацию от Сберлогистики, с понедельника жду информацию от Вас и от официального представителя Сбербанка. Я уже подсказала приблизительный план поиска посылки. Передайте, пожалуйста, его сотрудникам, ответственным за подразделение Сберлогистики. Официально предлагаю Вам мою помощь, могу стать волонтёром на пару дней и сама поискать свою посылку на складе.'},
    {'user_name':'luiza mariam_',
    'comment':'я понимаю, что других новостей нет. Но Вы не новостная программа, чтобы сообщать новости. Вы должны закрепить ответственного сотрудника за номером обращения и искать, искать посылку. Отправить заявку на первый склад, какой машиной и когда была отправлена посылка, пришла ли данная машина на второй склад, если пришла, то выяснить почему не были выгружены посылки, если были выгружены, то найти сотрудника, кто их принимал. Искать посылку на втором складе, а не ждать, когда сама посылка прыгнет какому-нибудь сотруднику в руки. Вещи стоят денег. Время стоит денег. А вы уже украли и мои вещи, и мои деньги, а теперь и время, которое я трачу на все эти разбирательства .'},
    {'user_name':'luiza mariam_',
    'comment':'прошу сказать мне, где находится моя посылка, которая уже 10 дней в Москве, хотя должна была быть в другом городе. Ответ ждите, мы ответим позже, у нас возможные задержки и прочие отписки это не ответ .'},
    {'user_name':'Иван Симон',
    'comment':'Люди жалуются на категории "пятёрочка", а у меня за 4 года пользования картой ни разу супермаркетов не было, а в основном категории которыми никогда не пользовался'},
    {'user_name':'Анастасия Ильина',
    'comment':'Не думала, что я вам это скажу, но я рада, что мои знакомые подключили в приложении сбп . Здесь меня держит только зарплата . Пользуюсь я другим банком . К сожалению , вы единственный банк , где сбп не подключена автоматически и мне приходится у людей спрашивать номер карты , так как в моем другом банке можно переводить на другие банки без комиссии даже по номеру карты . Единственный такой банк из всех . Поэтому я все время радуюсь , что можно переводить кому-то по номеру телефона в Сбербанк из другого банка без комиссии , когда подключена данная функция .'},
    {'user_name':'Максим Носов',
    'comment':'Банк мне понравился, стал клиентом после акции карта с бесплатным обслуживанием навсегда и получил уже хорошие бонусы для себя , даже сегодня прислали 500 р , там у них ним много програм каких-то , в этом надо разбираться , всем удачи'}
]

In [11]:
%time
for comment in test_comments:
    emotion_color = emotion_recognizer(comment['comment'])
    print('='*15)
    print('{}\n{}\n Оценён, как: {}'.format(comment['user_name'], comment['comment'], emotion_color))

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 15.7 µs
Gvido
Помогите Автономные тесты. Интеграционные, функциональные, unit-тесты Все давно уже знают, что такое MVC и согласны с тем, что это хорошо. Но откуда тогда берутся эти проекты с километровым кодом в контроллерах, да ещё и напрямую изменяющим БД? Почему это плохо и как должно быть, а также о главной боли Django — в этом выпуске.Посоветуйте курс по пайтону, реальный - не поверхностный. Реально хочу погрузиться в программирование. В этом видео мы реализуем Телеграм бота для учёта личных расходов с бюджетом, категориями расходов и базовой статистикой. Использовать будем Python и Aiogram с Asyncio, в качестве базы данных легковесный движок SQLite, а результат работы упакуем в Docker контейнер для быстрого разворачивания на любом сервере
 Оценён, как: neutral
luiza mariam_
я с прошлой субботы жду информацию от Сберлогистики, с понедельника жду информацию от Вас и от официального представителя Сбербанка. Я уже подсказала при

##### Само распознавание стреляет невпопад. Дальше надо выбрать метод более точного распознавания и орабатывать на комментариях

#### написать обработчик входящего текста, чтобы убирал ненужные предлоги, союзы, отрезал окончания, суффиксы, приставки (когда надо). Учитывал слово не и менял бы окрас слова на противоположный

In [1]:
import nltk

In [2]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/dima/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords

In [9]:
from string import punctuation

In [6]:
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [8]:
!poetry add pymystem3

The following packages are already present in the pyproject.toml and will be skipped:

  • pymystem3

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [11]:
from pymystem3 import Mystem

ModuleNotFoundError: No module named 'pymystem3'

In [12]:
text = '''Люди жалуются на категории "пятёрочка", а у меня за 4 года пользования картой ни разу супермаркетов не было, а в основном категории которыми никогда не пользовался'''

In [ ]:
from pymystem3 import Mystem
text = "Красивая мама красиво мыла раму"
m = Mystem()
lemmas = m.lemmatize(text)
print(''.join(lemmas))

#красивый мама красиво мыть рама

#Getting grammatical information and lemmas.

import json
from pymystem3 import Mystem

text = "Красивая мама красиво мыла раму"
m = Mystem()
lemmas = m.lemmatize(text)

print "lemmas:", ''.join(lemmas)
print "full info:", json.dumps(m.analyze(text), ensure_ascii=False, encoding='utf8')

#lemmas: красивый мама красиво мыть рама

#full info: [{"text": "Красивая", "analysis": [{"lex": "красивый", "gr": "A=им,ед,полн,жен"}]}, {"text": " "}, {"text": "мама", "analysis": [{"lex": "мама", "gr": "S,жен,од=им,ед"}]}, {"text": " "}, {"text": "красиво", "analysis": [{"lex": "красиво", "gr": "ADV="}]}, {"text": " "}, {"text": "мыла", "analysis": [{"lex": "мыть", "gr": "V,несов,пе=прош,ед,изъяв,жен"}]}, {"text": " "}, {"text": "раму", "analysis": [{"lex": "рама", "gr": "S,жен,неод=вин,ед"}]}, {"text": "\n"}]
#берём наречия и прилагательные

In [33]:
def preprocessor(text):
    text = text.lower()
    text = text.strip(',').split(' ')
    text = list(set(text) - stop_words)
    for word in text:
        print(word)
    #return text

In [13]:
#stop_words = '''без, безо, близ,в, во, вместо, вне для, до, за, из, изо, из-за, из за, из-под, к, ко, кроме, между, на, над, о, об, обо, от, ото, перед, передо, пo, под, подо, при, про, ради, с, со, сквозь, среди, у, через, чрез, и, а, но, да, если, что, когд, потому что, так как, как будто, вследствие того что, с тех пор как, в то время как, для того чтобы, ни – ни, то – то, не то – не то, то ли – то ли, а, но, зато, да, когда, пока, едва, лишь, лишь только, как только, потому что, оттого что, так как, ибо, так что, что, будто, чтобы, как'''

In [14]:
#stop_words = set(stop_words.split(', '))